<a href="https://colab.research.google.com/github/lucasresende1002/SafeSport/blob/main/risco_de_lesao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
import joblib

In [ ]:
np.random.seed(42)
n = 5000

df = pd.DataFrame({
    'treino_duracao': np.random.randint(40, 120, n),             # minutos de treino
    'freq_cardiaca': np.random.randint(120, 190, n),             # bpm médio
    'PSE': np.random.randint(4, 10, n),                          # percepção subjetiva de esforço (0-10)
    'dist_percorrida': np.random.uniform(3, 12, n).round(2),     # distância percorrida (km)
    'dias_desde_ultima_lesao': np.random.randint(0, 60, n),      # dias desde última lesão
})

# 1. Definir a probabilidade de lesão com base nos fatores de risco conhecidos
# Esses fatores foram retirados de "padrões realistas achados nas pesquisas realizadas pela UFMG e USP"
# A probabilidade de lesão não é mais 0 ou 1 rígido.
prob_lesao_base = (
    (df['freq_cardiaca'] > 165).astype(float) * 0.35 +
    (df['PSE'] > 7).astype(float) * 0.30 +
    (df['dias_desde_ultima_lesao'] < 15).astype(float) * 0.20 +
    0.05
)

# 2. Ajuste de probabilidade
prob_lesao_ajustada = np.clip(prob_lesao_base, 0.05, 0.95)

# 3. A lesão ocorre se um número aleatório (entre 0 e 1) for menor que a probabilidade calculada
df['lesao'] = (np.random.rand(n) < prob_lesao_ajustada).astype(int)

# -----------------------------------------------------------------------------------------

X = df.drop('lesao', axis=1)
y = df['lesao']

print("Visualização inicial dos dados:")
print(df.head())
print(f"\nProporção de lesões (1s): {df['lesao'].mean()*100:.2f}%")

Visualização inicial dos dados:
   treino_duracao  freq_cardiaca  PSE  dist_percorrida  \
0              91            137    8            10.86   
1              54            188    5             6.36   
2             111            173    9            10.57   
3             100            149    5             6.88   
4              60            164    7             6.61   

   dias_desde_ultima_lesao  lesao  
0                        5      0  
1                       31      0  
2                        1      0  
3                       23      0  
4                       44      0  

Proporção de lesões (1s): 32.80%


In [ ]:
# Pré-processamento de dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar os dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Modelo de Treinamento
modelo = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
modelo.fit(X_train_scaled, y_train)

GradientBoostingClassifier(random_state=42)

In [ ]:
# Avaliação do modelo
y_pred = modelo.predict(X_test_scaled)
y_prob = modelo.predict_proba(X_test_scaled)[:, 1]

print("\n--- Resultados do Modelo ---")
print("Acurácia:", accuracy_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))


--- Resultados do Modelo ---
Acurácia: 0.746
ROC-AUC: 0.8112217395440275

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.78      0.86      0.82       671
           1       0.65      0.50      0.57       329

    accuracy                           0.75      1000
   macro avg       0.71      0.68      0.69      1000
weighted avg       0.74      0.75      0.74      1000



In [ ]:
# Salvar modelo e scaler
joblib.dump(modelo, 'modelo_lesao.pkl')
joblib.dump(scaler, 'scaler.pkl')
print("\nModelo e Scaler salvos como 'modelo_lesao.pkl' e 'scaler.pkl'.")


Modelo e Scaler salvos como 'modelo_lesao.pkl' e 'scaler.pkl'.


In [ ]:
def prever_lesao(treino_duracao, freq_cardiaca, PSE, dist_percorrida, dias_desde_ultima_lesao):

    # Carregar modelo e scaler salvos
    try:
        modelo = joblib.load('modelo_lesao.pkl')
        scaler = joblib.load('scaler.pkl')
    except FileNotFoundError:
        return "Erro: Arquivos do modelo não encontrados. Execute o treinamento primeiro.", ""

    # Montar os dados do atleta
    dados = pd.DataFrame([[treino_duracao, freq_cardiaca, PSE, dist_percorrida, dias_desde_ultima_lesao]],
                         columns=["treino_duracao", "freq_cardiaca", "PSE", "dist_percorrida", "dias_desde_ultima_lesao"])

    # Normalizar com o mesmo padrão do treino
    dados_scaled = scaler.transform(dados)

    # Fazer a previsão (probabilidade da classe 1 - lesão)
    prob_lesao = modelo.predict_proba(dados_scaled)[0][1]

    # Classificar risco
    if prob_lesao < 0.3:
        risco = "🟢 Baixo (Pode seguir o treino planejado)"
    elif prob_lesao < 0.6:
        risco = "🟡 Moderado (Recomendada cautela e monitoramento)"
    else:
        risco = "🔴 Alto (Sugestão de modificação ou pausa no treino)"

    return prob_lesao, risco

In [ ]:
def menu():
    print("\n--- Sistema de Previsão de Risco de Lesão ---")

    while True:
        print("\n1 - Inserir dados do atleta")
        print("2 - Sair")
        opcao = input("Escolha uma opção: ")

        if opcao == "1":
            try:
                treino_duracao = float(input("Duração do treino (minutos): "))
                freq_cardiaca = float(input("Frequência cardíaca média (bpm): "))
                PSE = float(input("PSE (0 a 10): "))
                dist_percorrida = float(input("Distância percorrida (km): "))
                dias_lesao = int(input("Dias desde a última lesão: "))

                prob, risco = prever_lesao(
                    treino_duracao=treino_duracao,
                    freq_cardiaca=freq_cardiaca,
                    PSE=PSE,
                    dist_percorrida=dist_percorrida,
                    dias_desde_ultima_lesao=dias_lesao
                )

                print(f"\n--- RESULTADO ---")
                print(f"Probabilidade de lesão: {prob * 100:.2f}%")
                print(f"Nível de risco: {risco}")

            except ValueError:
                print("\n⚠️ Erro: Digite apenas números válidos!")

        elif opcao == "2":
            print("\nEncerrando o sistema. Até mais!")
            break
        else:
            print("\n⚠️ Opção inválida! Tente novamente.")
menu()


--- Sistema de Previsão de Risco de Lesão ---

1 - Inserir dados do atleta
2 - Sair
Escolha uma opção: 1
Duração do treino (minutos): 120
Frequência cardíaca média (bpm): 100
PSE (0 a 10): 6
Distância percorrida (km): 10
Dias desde a última lesão: 45

--- RESULTADO ---
Probabilidade de lesão: 16.48%
Nível de risco: 🟢 Baixo (Pode seguir o treino planejado)

1 - Inserir dados do atleta
2 - Sair
Escolha uma opção: 2

Encerrando o sistema. Até mais!
